# Project Stage 3: Blocking Step

Problem Description:
We have extracted two tables A, B. Both tables have the same schema. Now we need to perform entity matching and will do so with Magellan, an EM tool to match the two tables. The first step is to perform blocking in order to reduce the number of potential matches considered by a classifier.


In [1]:
import py_entitymatching as em
import pandas as pd
import os, sys
import numpy as np

# Read CSV

- Read csv file from disk as a table and set its metadata

In [2]:
metacriticData = pd.read_csv('data/metacritic.csv')
wikiData = pd.read_csv('data/wikiData.csv')

In [3]:
# add ID column to each dataset
metacriticID = ["a" + str(num) for num in np.arange(1, len(metacriticData.index)+1)]
wikiID = ["b" + str(num) for num in np.arange(1, len(wikiData.index)+1)]

col_idx = 0
metacriticData.insert(loc = col_idx, column = 'ID', value = metacriticID)
wikiData.insert(loc = col_idx, column = 'ID', value = wikiID)

In [4]:
metacriticData.head()

,ID,Album,Artist,Genre,Label,Producer,Release Date,Meta Score
0,a1,Wrong Creatures,Black Rebel Motorcycle Club,['Pop/Rock'],['Vagrant Records'],NaN,Jan 12 2018,69
1,a2,No Cross No Crown,Corrosion of Conformity,['Pop/Rock'],['Nuclear Blast'],NaN,Jan 12 2018,77
2,a3,Encore,Anderson East,['Singer-Songwriter'],['Low Country Sound'],NaN,Jan 12 2018,74
3,a4,A Day With The Homies [EP],Panda Bear,['Pop/Rock'],['Domino'],NaN,Jan 12 2018,74
4,a5,Four Stones,Dean McPhee,['Alternative'],['Hood Faire'],NaN,Jan 12 2018,84


In [5]:
wikiData.head()

,ID,Album,Artist,Genre,Label,Producer,Release Date,Meta Score
0,b1,Be Calm,Air Dubai,"['Hip hop', ' pop']",['Hopeless'],"['Dwight A. Baker', ' Colin Munroe']",Jul 1 2014,NaN
1,b2,From Parts Unknown,Every Time I Die,"['Metalcore', ' hardcore punk', ' mathcore', ' sludge metal']",['Epitaph'],['Kurt Ballou'],Jul 1 2014,NaN
2,b3,"I'm Almost Happy Here, But I Never Feel At Home",Hotel Books,"['Spoken word', ' indie rock', ' emo']",['inVogue'],"['Jay Maas', ' Hiram Hernandez']",Jul 1 2014,NaN
3,b4,Paula,Robin Thicke,['R&B'],"['Star Trak', ' Interscope']","['Robin Thicke', ' Pro Jay']",Jul 1 2014,NaN
4,b5,Isolate and Medicate,Seether,"['Post-grunge', ' hard rock', ' alternative metal']","['The Bicycle Music Company', ' Concord Bicycle', ' Spinefarm']","[""Brendan O'Brien""]",Jul 1 2014,NaN


In [6]:
# set metadata
em.set_key(wikiData, 'ID')
em.set_key(metacriticData, 'ID')

#set albums and artists to lower case
wikiData["Album"] = wikiData["Album"].str.lower()
metacriticData["Album"] = metacriticData["Album"].str.lower()

wikiData["Artist"] = wikiData["Artist"].str.lower()
metacriticData["Artist"] = metacriticData["Artist"].str.lower()

#preprocessing -- set all months to have only 3 letters so dates match
def shorten_month(x):
    month, day, year = x.split(" ")
    month = month[0:3]
    return " ".join([month,day,year])
wikiData["Release Date"] = wikiData["Release Date"].apply(shorten_month)
metacriticData["Release Date"] = metacriticData["Release Date"].apply(shorten_month)

# Blocking
We will union the results of 4 blockers to get a final set of candidates. 
1. Attribute equivalence on album title
2. Attribute equivalence on artist
3. Attribute equivalence on release date and at least 2 word overlap on album title
4. Attribute equivalence on release date and at least 1 word overlap on artist

In [7]:
#1. block with album title
ab1 = em.AttrEquivalenceBlocker()
C1 = ab1.block_tables(metacriticData, wikiData, 
                   l_block_attr='Album', r_block_attr='Album', 
                    l_output_attrs=['Album', 'Artist', 'Release Date'],
                    r_output_attrs=['Album', 'Artist', 'Release Date'])

print len(C1)
C1.head(5)

1034


,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Release Date
0,0,a3,b1611,encore,anderson east,Jan 12 2018,encore,dj snake,Aug 5 2016
1,1,a1783,b1611,encore,dj snake,Aug 5 2016,encore,dj snake,Aug 5 2016
2,2,a9,b746,underworld,tonight alive,Jan 12 2018,underworld,symphony x,Jul 24 2015
3,3,a16,b3181,saturation iii,brockhampton,Dec 15 2017,saturation iii,brockhampton,Dec 15 2017
4,4,a18,b933,revival,eminem,Dec 15 2017,revival,selena gomez,Oct 9 2015


In [8]:
#2. block with artist name
ab2 = em.AttrEquivalenceBlocker()
C2 = ab2.block_tables(metacriticData, wikiData, 
                   l_block_attr='Artist', r_block_attr='Artist', 
                    l_output_attrs=['Album', 'Artist', 'Release Date'],
                    r_output_attrs=['Album', 'Artist', 'Release Date'],
                    allow_missing=True)
print len(C2)
C2.head(5)

2100


,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Release Date
0,0,a4,b359,a day with the homies [ep],panda bear,Jan 12 2018,panda bear meets the grim reaper,panda bear,Jan 13 2015
1,1,a8,b249,vale,black veil brides,Jan 12 2018,black veil brides,black veil brides,Oct 27 2014
2,2,a9,b1263,underworld,tonight alive,Jan 12 2018,limitless,tonight alive,Mar 4 2016
3,3,a2369,b1263,limitless,tonight alive,Mar 4 2016,limitless,tonight alive,Mar 4 2016
4,4,a11,b938,blue madonna,bØrns,Jan 12 2018,dopamine,bØrns,Oct 16 2015


In [9]:
#3. Overlap
#3.1 overlap at least 2 words of album title
ob3 = em.OverlapBlocker()
C3 = ob3.block_tables(metacriticData, wikiData,'Album','Album',word_level=True,overlap_size=2,
                   l_output_attrs=["Album","Artist","Release Date"],
                   r_output_attrs=["Album","Artist","Release Date"],
                    show_progress=False)

#3.2. same release date
a3 = em.AttrEquivalenceBlocker()
C3 = a3.block_candset(C3, 'Release Date', 'Release Date',show_progress=False)
print(len(C3))

812


In [10]:
#4. Overlap
#4.1 overlap at least 1 words of artist
ob4 = em.OverlapBlocker()
C4 = ob4.block_tables(metacriticData, wikiData,'Artist','Artist',word_level=True,overlap_size=1,
                   l_output_attrs=["Album","Artist","Release Date"],
                   r_output_attrs=["Album","Artist","Release Date"],
                    show_progress=False)

#4.2. same release date
a4 = em.AttrEquivalenceBlocker()
C4 = a4.block_candset(C4, 'Release Date', 'Release Date',show_progress=False)
print(len(C4))

1595


### Union results together to get final candidate set

In [11]:
G = em.combine_blocker_outputs_via_union([C1, C2, C3, C4])
print len(G)

G.head(10)

3006


,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Release Date
0,0,a1001,b2328,black and white rainbows,bush,Mar 10 2017,black and white rainbows,bush,Mar 10 2017
1,1,a1001,b238,black and white rainbows,bush,Mar 10 2017,man on the run,bush,Oct 21 2014
2,2,a1004,b2330,different creatures,circa waves,Mar 10 2017,different creatures,circa waves,Mar 10 2017
3,3,a1004,b494,different creatures,circa waves,Mar 10 2017,young chasers,circa waves,Mar 30 2015
4,4,a1005,b2355,jawbone [original motion picture soundtrack],paul weller,Mar 10 2017,beauty and the beast: original motion picture soundtrack,various artists,Mar 10 2017
5,5,a1005,b2675,jawbone [original motion picture soundtrack],paul weller,Mar 10 2017,a kind revolution,paul weller,May 12 2017
6,6,a1009,b2350,elektrac,shobaleader one,Mar 10 2017,elektrac,shobaleader one,Mar 10 2017
7,7,a1010,b2358,packs,your old droog,Mar 10 2017,packs,your old droog,Mar 10 2017
8,8,a1010,b324,packs,your old droog,Mar 10 2017,your old droog,your old droog,Nov 25 2014
9,9,a1012,b2346,captain california,murs,Mar 10 2017,captain california,murs,Mar 10 2017


### Run debugger to make sure not missing any valid matches

In [12]:
dbg = em.debug_blocker(G,metacriticData,wikiData,output_size=50)
dbg

,_id,ltable_ID,rtable_ID,ltable_Album,ltable_Artist,ltable_Genre,ltable_Label,ltable_Producer,ltable_Release Date,rtable_Album,rtable_Artist,rtable_Genre,rtable_Label,rtable_Producer,rtable_Release Date
0,0,a698,b2713,a world of masks,the heliocentrics,['Electronic'],['Soundway'],NaN,May 26 2017,in a world of fear,scale the summit,NaN,NaN,NaN,May 19 2017
1,1,a2144,b1976,prayers for the damned,sixx: a.m.,['Pop/Rock'],['Eleven Seven'],NaN,Apr 29 2016,prayers for the blessed,sixx:a.m.,NaN,['Eleven Seven'],NaN,Nov 18 2016
2,2,a1746,b2964,a weird exits,thee oh sees,['Pop/Rock'],['Castle Face'],NaN,Aug 12 2016,orc,oh sees,NaN,['Castle Face'],NaN,Aug 25 2017
3,3,a1822,b2819,vulnicura live,björk,['Electronic'],['One Little Indian'],NaN,Jul 22 2016,songs from the novel 'greatest hits',kathryn williams,NaN,['One Little Indian'],['Romeo Stodart'],Jun 16 2017
4,4,a1768,b101,blossoms,blossoms,['Pop/Rock'],['Virgin EMI'],NaN,Aug 5 2016,listen,the kooks,['Indie rock'],['Virgin EMI'],"['Luke Pritchard', ' Inflo', ' Fraser T Smith']",Sep 1 2014
5,5,a1790,b254,for all we know,nao,['R&B'],['RCA'],NaN,Jul 29 2016,honor is all we know,rancid,"['Punk rock', ' ska punk']","['Hellcat', ' Epitaph']",['Brett Gurewitz'],Oct 27 2014
6,6,a1804,b1765,live in san francisco,thee oh sees,['Pop/Rock'],['Castle Face'],NaN,Jul 29 2016,live in san diego,eric clapton,"['Rock', ' blues rock', ' pop rock']",['Reprise'],"['Simon Climie', ' Eric Clapton']",Sep 30 2016
7,7,a1804,b2964,live in san francisco,thee oh sees,['Pop/Rock'],['Castle Face'],NaN,Jul 29 2016,orc,oh sees,NaN,['Castle Face'],NaN,Aug 25 2017
8,8,a1809,b532,born of the sun,faun fables,['Folk'],['Drag City'],NaN,Jul 22 2016,edge of the sun,calexico,"['Alt-rock', ' indie folk', ' Americana', ' Tex-Mex', ' alt-country', ' post-rock']",['ANTI-'],"['Joey Burns', ' John Convertino', ' Sergio Mendoza', ' Craig Schumacher']",Apr 13 2015
9,9,a1814,b1131,redshift,rhyton,['Pop/Rock'],['Thrill Jockey'],NaN,Jul 22 2016,the catastrophist,tortoise,['Post-rock'],['Thrill Jockey'],NaN,Jan 22 2016


### Save dataset and set aside sample for labeling

In [14]:
# save data
file_name = 'candidates.csv'
G.to_csv(file_name, sep=',')

# Take sample to label
s_file_name = "candidates_sample.csv"
import os.path
if not os.path.isfile(s_file_name): #so you don't delete your labels on accident
    S = em.sample_table(G,100)
    S.to_csv(s_file_name,sep=",")
else:
    print "Sample file already exists, so not overwriting it"